In [1]:
import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt

# handle exception trace for debugging 
# background loop
import traceback

import time
import random

In [2]:
cam = py.InstantCamera(py.TlFactory.GetInstance().CreateFirstDevice())
cam.Open()

# to get consistant results it is always good to start from "power-on" state
cam.UserSetSelector = "Default"
cam.UserSetLoad.Execute()

cam.ExposureTime = cam.ExposureTime.Min

In [3]:
# show expected framerate max framerate ( @ minimum exposure time)
cam.ResultingFrameRate.Value

156.20118712902217

In [4]:
# this results in frame period in µs
1 / cam.ResultingFrameRate.Value * 1e6

6402.000000000001

we now compare different pylon grab strategies and their resulting frameperiod to this minimum frameperiod

# Grab scenarios

## GrabOne Loop

most simple style to grab

In [5]:
def GrabOneSample():
    # fetch some images with foreground loop
    img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)

    for i in range(100):
        with cam.GrabOne(1000) as res:
            img = res.Array
            img_sum += img
    return img_sum

In [6]:
%%timeit -o
GrabOneSample()

3.27 s ± 41.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 3.27 s ± 41.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [7]:
grab_one_average = _.average / 100

In [8]:
f"time to capture one frame is: {grab_one_average*1e6:.2f} µs"

'time to capture one frame is: 32655.76 µs'

this is very easy to use, but will have the overhead of starting and stopping the grab engine for every frame

## Foreground Loop

move the start and stop of the grab engine out of the inner grab loop

In [9]:
def ForegroundLoopSample():
    # fetch some images with foreground loop
    img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)
    cam.StartGrabbingMax(100)
    while cam.IsGrabbing():
        with cam.RetrieveResult(1000) as res:
            if res.GrabSucceeded():
                img = res.Array
                img_sum += img
            else:
                raise RuntimeError("Grab failed")
    cam.StopGrabbing()
    return img_sum

In [ ]:
%%timeit -o 
ForegroundLoopSample()

In [ ]:
foreground_average = _.average / 100

In [ ]:
f"average time to capture one frame is: {foreground_average*1e6:.2f} µs"

## Background loop

* use this mode to communicate with the camera while the images are collected in the backgound.
* Allows async communication

Pylon uses classes with callback handler to let usercode communicate with the grab loop

In [ ]:
class ImageHandler(py.ImageEventHandler):
    def __init__(self):
        super().__init__()
        self.img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)
        
    def OnImageGrabbed(self, camera, grabResult):
        """ we get called on every image
            !! this code is run in a pylon thread context
            always wrap your code in the try .. except to capture
            errors inside the grabbing as this can't be properly reported from 
            the background thread to the foreground python code
        """
        try:
            if grabResult.GrabSucceeded():
                # check image contents
                img = grabResult.Array
                self.img_sum += img
            else:
                raise RuntimeError("Grab Failed")
        except Exception as e:
            traceback.print_exc()

In [ ]:
def BackGroundLoopSample():

    # instantiate callback handler
    handler = ImageHandler()
    # register with the pylon loop
    cam.RegisterImageEventHandler( handler , py.RegistrationMode_ReplaceAll, py.Cleanup_None)

    # fetch some images with background loop
    cam.StartGrabbingMax( 100, py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)
    while cam.IsGrabbing():
        # random exposuretime changes every 100ms
        cam.ExposureTime = random.uniform(cam.ExposureTime.Min, 1000)
        time.sleep(0.1)

    cam.StopGrabbing()
    cam.DeregisterImageEventHandler(handler)
    
    return handler.img_sum

In [ ]:
%%timeit -o
BackGroundLoopSample()

In [ ]:
background_average = _.average / 100

In [ ]:
f"average time to capture one frame is: {background_average*1e6:.2f} µs"

In [ ]:
cam.StopGrabbing()

In [ ]:
cam.Close()